In [2]:
import numpy as np # it is a numerical computing library and its known for its fast excuetion speed, it's C++ module
import pandas as pd # it makes every easy to work with tabular data in python
import requests #popular library which is considered as a gold standard for making HTTP requests(Internet Request send to an API)
import xlsxwriter # write in excel documents
import math

In [3]:
stocks = pd.read_csv(r'G:\Algorihtmic Trading\algorithmic-trading-python-master\starter_files\sp_500_stocks.csv')


In [4]:
from secrets_1 import IEX_CLOUD_API_TOKEN

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'MNSTCLSQEEBORTKENAAGAGLA( D/ ) LS', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'folciaif', 'close': None, 'closeTime': None, 'closeSource': 'filofaic', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 134.65, 'latestSource': 'Close', 'latestTime': 'April 20, 2021', 'latestUpdate': 1635661510867, 'latestVolume': None, 'iexRealtimePrice': 134.99, 'iexRealtimeSize': 46, 'iexLastUpdated': 1678507497055, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 137.7, 'previousVolume': 97682370, 'change': -1.73, 'changePercent': -0.01326, 'volume': None, 'iexMarketPercent': 117.89044418836617, 'iexVolume': 1723590, 'avgTotalVolume': 98462940, 'iexBidPrice': 0

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']

In [7]:
my_columns = ['Ticker','Stock Price','Market Captilization','Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Captilization,Number of Shares to Buy


In [8]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index = True
)

,Ticker,Stock Price,Market Captilization,Number of Shares to Buy
0,AAPL,134.65,2275911945545,N/A


In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
smaller_groups = list(chunks(stocks['Ticker'], 100))
symbol_string = []
for i in range(0,len(smaller_groups)):
    symbol_string.append(','.join(smaller_groups[i]))
#    print(symbol_string)
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_s in symbol_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_s}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_s.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ], 
            index = my_columns), 
            ignore_index = True
        )
        
    
final_dataframe

,Ticker,Stock Price,Market Captilization,Number of Shares to Buy
0,A,134.54,40638116863,N/A
1,AAL,20.61,13468201008,N/A
2,AAP,198.99,13171247507,N/A
3,AAPL,135.04,2332084304747,N/A
4,ABBV,112.30,201167567520,N/A
...,...,...,...,...
500,YUM,121.99,36442829565,N/A
501,ZBH,177.42,36487375168,N/A
502,ZBRA,514.58,27066479528,N/A
503,ZION,53.50,8926006736,N/A


In [11]:
portfolio_size = input("Enter the value of your potfolio : ")

try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \n Please try again:")
    portfolio_size = input("Enter the value of your potfolio : ")
    val = float(portfolio_size)

10000000.0


In [12]:
position_size = val/len(final_dataframe.index)
#number_of_Apple_shares = position_size / 500
#print(math.floor(number_of_Apple_shares))
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
    
final_dataframe

,Ticker,Stock Price,Market Captilization,Number of Shares to Buy
0,A,134.54,40638116863,147
1,AAL,20.61,13468201008,960
2,AAP,198.99,13171247507,99
3,AAPL,135.04,2332084304747,146
4,ABBV,112.30,201167567520,176
...,...,...,...,...
500,YUM,121.99,36442829565,162
501,ZBH,177.42,36487375168,111
502,ZBRA,514.58,27066479528,38
503,ZION,53.50,8926006736,370


In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [16]:
writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)
# writer.save()

0

In [17]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [18]:
writer.save()